In [36]:
import pandas as pd
import folium
from folium import plugins
from folium import IFrame
from folium.features import DivIcon

In [42]:
# Create a map centered in Lisbon
m = folium.Map(location=[38.736946, -9.142685], zoom_start=13, control_scale=True, tiles=None)


# Create a feature group for basemaps
fg1 = folium.FeatureGroup(name='OpenStreetMap', overlay=False)
folium.TileLayer(tiles='OpenStreetMap', name='OpenStreetMap').add_to(fg1)

# Add satellite imagery basemap
fg2 = folium.FeatureGroup(name="Esri World Imagery", overlay=False)
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite'
).add_to(fg2)

# Add feature group with basemaps to the map
m.add_child(fg1)
m.add_child(fg2)

# Add LayerControl to toggle between basemaps
folium.LayerControl(position='bottomleft').add_to(m)


# Load the CSV file containing the coordinates
df = pd.read_csv('BAP_Image_table.csv', sep=',')


for index, row in df.iterrows():
    # Assuming you have a way to construct the URL for each image
    image_url = f"https://emocities.s3.eu-central-1.amazonaws.com/{row['REFERENCE']}.jpg"
    
    # Custom HTML content for the DivIcon
    html = f'<img src="{image_url}" style="width: 50px; height: auto; border: 2px solid white; border-radius:7px;">'  # Small thumbnail size
    
    # Create an icon with the custom HTML
    icon = DivIcon(
        icon_size=(25,25),  # Size of the icon
        icon_anchor=(25,25),  # Anchor point
        html=html,
    )

    popup_html = folium.Html(f"<img src='{image_url}' width='450'>", script=True)
    popup = folium.Popup(popup_html, max_width=2650)
    
    # Add this icon to the map without a marker icon
    folium.Marker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        icon=icon,
        popup=popup
    ).add_to(m)

mapFname = 'Output_Lisbon_Img.html'
m.save(mapFname)